# Neural Network

Let's try to "learn" the "exclusive or" function. Unlike "and" or "or", it cannot be learned by a single neuron/perceptron, because it is not linearly seperable.

In [1]:
import numpy as np